In [1]:
# Using https://stackoverflow.com/questions/67268723/insert-json-data-into-postgres-table-using-python as guide
import requests, json
import psycopg2
import pandas as pd
import config

db_password = config.password

In [2]:
# define url for api call
sales_url = "https://api.guildwars2.com/v2/commerce/transactions/history/sells?access_token=900E00C1-390E-3143-B44A-D8EF1E9224195B43D84C-9F9E-4669-A538-7FF4A5DA70C1"
# make api call
response = requests.get(sales_url)

In [3]:
# get items endpoint from Silveress' API
items_url = 'https://api.datawars2.ie/gw2/v1/items/json/new'
# make API call and get response, converting to json
items_response_json = requests.get(items_url).json()

In [4]:
# Create empty list to store sell prices and replace any values that are missing (ie items in GW2 that aren't sold on TP) with 0
sell_prices = []
for i in range(len(items_response_json)):
    try:
        sell_prices.append(items_response_json[i]['sell_price'])
    except KeyError:
        items_response_json[i]['sell_price'] = 0
        items_response_json[i]['sell_quantity'] = 0
        items_response_json[i]['buy_price'] = 0
        items_response_json[i]['buy_quantity'] = 0

In [5]:
# get api call from GW2 sales endpoint response as json (list of dicts)
sales_response_json = response.json()

In [6]:
# make connection to postgres database
con = psycopg2.connect(f'dbname=gw2_bltc user=postgres host=localhost password={db_password}')
# instantiate the cursor for which we execute with
cur = con.cursor()
# create two INSERT INTO queries, using string formtting to replace these with values from our JSON
insert_sql_sales = """INSERT INTO sales VALUES(%s, %s, %s, %s, %s, %s) ON CONFLICT (id) DO NOTHING;"""
insert_sql_items = """INSERT INTO item_dict VALUES(%s, %s, %s, %s, %s, %s, %s) ON CONFLICT (id) DO NOTHING;"""

for record in sales_response_json:
    cur.execute(insert_sql_sales, [
        record['id'], record['item_id'], record['price'], record['quantity'],
        record['created'], record['purchased']
    ])
con.commit()

for record in items_response_json:
    cur.execute(insert_sql_items, [
        record['id'], record['buy_price'], record['buy_quantity'],
        record['lastUpdate'], record['name'], record['sell_price'],
        record['sell_quantity']
    ])
con.commit()